In [18]:
# ===Check the uniqueness of subject_is in each file===

# import pandas library
import pandas as pd

# List of file names in problem set
files = ['woodlawn.csv', 'hyde_park.csv', 'outcome_data.csv', 'treatment.csv']

# using for loop, read the file one by one.
for file in files:
    # Load the dataset
    df = pd.read_csv(file)
        
    # use .is_unique to check values of subject_id are unique, and show the result with the numebr of observation in each file.
    print("-" * 30)
    print(f"N of {file} ",len(df),)
    print(file, df['subject_id'].is_unique)
    

------------------------------
N of woodlawn.csv  476
woodlawn.csv True
------------------------------
N of hyde_park.csv  497
hyde_park.csv True
------------------------------
N of outcome_data.csv  989
outcome_data.csv False
------------------------------
N of treatment.csv  1004
treatment.csv False


In [19]:
# ===Check obsevations with non-unique subject id ====

# import pandas library
import pandas as pd

# List of file names which cantain dupulicate ids
files_to_check = ['outcome_data.csv', 'treatment.csv']

for file in files_to_check:
    df = pd.read_csv(file)
    
    #  Extract all rows with duplicate subject_ids
    # Setting keep=False displays all occurrences, including the first one
    duplicates = df[df.duplicated(subset=['subject_id'], keep=False)]
    
    # Sorting by ID 
    duplicates = duplicates.sort_values(by='subject_id')
    
    print(f"--- Duplicates subject_id of {file}  ---")
    print(duplicates)

--- Duplicates subject_id of outcome_data.csv  ---
     subject_id  hours
243         NaN    NaN
779         NaN    NaN
--- Duplicates subject_id of treatment.csv  ---
      subject_id  treatment
32            33          0
1000          33          0
33            34          1
1001          34          1
57            58          0
1002          58          0
1003          58          0


In [23]:
# ====Check whether each file includes missing values====

# import pandas library
import pandas as pd

# List of target files
files = ['woodlawn.csv', 'hyde_park.csv', 'outcome_data.csv', 'treatment.csv']

for file in files:
    df = pd.read_csv(file)
    
    print(f"--- Missing Values in {file} ---")
    
    # Identify missing values with isnull() and calculate the sum per column
    missing_counts = df.isnull().sum()
    
    # Filter and display only columns that have at least one missing value for better readability
    print(missing_counts[missing_counts > 0])
    

--- Missing Values in woodlawn.csv ---
race_ethnicity    10
dtype: int64
--- Missing Values in hyde_park.csv ---
race_ethnicity    17
dtype: int64
--- Missing Values in outcome_data.csv ---
subject_id    2
hours         2
dtype: int64
--- Missing Values in treatment.csv ---
Series([], dtype: int64)


In [25]:
# ====Cleaning dataset_1:drop obsevations with non-unique id====

# import pandas library
import pandas as pd

# List of datasets which has non unique id
files = ['outcome_data.csv', 'treatment.csv']

print(f"{'File Name':<20} | {'Before':<10} | {'After':<10} | {'Removed'}")
print("-" * 55)

# Using for loop, drop obsevations with non-unique subject id in each file. 
for file in files:
    # Load the original data
    df = pd.read_csv(file)
    initial_n = len(df)
    
    # Cleaning Step 1: Remove missing subject_ids
    df_no_na = df.dropna(subset=['subject_id'])
    
    # Cleaning Step 2: Remove duplicate subject_ids
    df_final = df_no_na.drop_duplicates(subset=['subject_id'], keep=False)
    final_n = len(df_final)
    
    # Calculate how many rows were deleted
    removed = initial_n - final_n
    
    # Print the results in a table format
    print(f"{file:<20} | {initial_n:<10} | {final_n:<10} | {removed}")

    df_final.to_csv(f"removed_non_unique_ID_{file}", index=False)

File Name            | Before     | After      | Removed
-------------------------------------------------------
outcome_data.csv     | 989        | 987        | 2
treatment.csv        | 1004       | 997        | 7


In [26]:
# ===intergarte "woodlawn" and "hyde_parkand" using subject_id as a key.===

# import pandas library
import pandas as pd

# 1. Load the community datasets
df_woodlawn = pd.read_csv('woodlawn.csv')
df_hyde_park = pd.read_csv('hyde_park.csv')

# 2. Load the pre-cleaned treatment and outcome datasets
df_treatment_cleaned = pd.read_csv('removed_non_unique_ID_treatment.csv')
df_outcome_cleaned = pd.read_csv('removed_non_unique_ID_outcome_data.csv')

# 3. Vertically stack the community center datasets (The "Master List")
df_community_combined = pd.concat([df_woodlawn, df_hyde_park])

# 4. Perform LEFT JOIN with treatment data
# 'how=left' ensures all observations in dataset of two comunnity centers are kept, even if missing in treatment
df_merged_step1 = pd.merge(df_community_combined, df_treatment_cleaned, on='subject_id', how='left')

# 5. Perform LEFT JOIN with outcome data
# This keeps the observation even if they didn't record any 'hours'
df_final_left_merged = pd.merge(df_merged_step1, df_outcome_cleaned, on='subject_id', how='left')

# 6. Export the final data (using utf-8-sig for Excel compatibility)
df_final_left_merged.to_csv('merged_left_data_removed_nonunique_id_not_clearned_forQ1Q2.csv', index=False, encoding='utf-8-sig')

# 7. Report the numbers
print(f"Total Rows: {len(df_final_left_merged)}")
print("Check Unique:",df_final_left_merged['subject_id'].is_unique)

Total Rows: 973
Check Unique: True


In [24]:
# ===Cleaning dataset_2: drop the observation with some missing values or invalid values===

# --prompt--
# 以下の手順で、Pythonコードを書いて。
# merged_left_data_removed_nonunique_id_not_clearned_forQ1Q2.csv を読み込んで。最初に Start N: [行数] を表示して。
# フィルタリング（各ステップで行数を表示して）
# 欠損値（NaN）がある行をすべて削除して。
# age がマイナスの行を削除して。
# hours が100を超える行を削除して。
# bmi が1.0未満の行を削除して。
# treatment が 0 と 1 以外の行を削除して。
# community_center 列を小文字にして空白を除去して。'woodlawn' は 0 に、'hyde park', 'hydepark', 'hyde_park', 'hyd park' はすべて 1 に変換して。
# female 列の "female" を 1、"male" を 0 に変換して。最終的に数値型（int）になるようにして。
# education と race_ethnicity はそれぞれ小文字にして空白を除去して。
# pd.get_dummies を使ってダミー変数を作って。結果が True/False ではなく 1 と 0（int型） になるようにして。
# 教育歴は "less than high school", "high school", "higher degree" の3列、人種は "white", "black", "hispanic" の3列を元のデータに結合して。
# 最後に subject_id の昇順でデータを並べ替えて。
#ファイル名を cleaned_data_forQ3_Q9.csv にして保存して。Excelで開いても列が分かれるように encoding='utf-8-sig' を使って、インデックスは含めないで。
# 最後に Final N: [行数] を表示して。
# ----

# import pandas library
import pandas as pd

# Opt-in to future Pandas behavior to suppress warnings when converting data types during replace operations
pd.set_option('future.no_silent_downcasting', True)

# Load the dataset
# Note: Ensure the file name matches exactly as provided
df = pd.read_csv('merged_left_data_removed_nonunique_id_not_clearned_forQ1Q2.csv')

# Initial observation count
print(f"Start N: {len(df)}")

# 1. Remove observations with missing values in any columns
df = df.dropna()
print(f"1. After removing missing values: {len(df)}")

# 2. Remove observations with negative age
df = df[df['age'] >= 0]
print(f"2. After removing negative age: {len(df)}")

# 3. Remove observations where hours are more than 100
df = df[df['hours'] <= 100]
print(f"3. After removing hours > 100: {len(df)}")

# 4. Remove observations where bmi is less than 1.0
df = df[df['bmi'] >= 1.0]
print(f"4. After removing BMI < 1.0: {len(df)}")

# 5. Remove observations where treatment is not 0 or 1
df = df[df['treatment'].isin([0, 1])]
print(f"5. After removing invalid treatment codes: {len(df)}")

# 6 & 7. Standardize Community Center names (Woodlawn -> 0, Hyde Park -> 1)
# Use .str.lower() for case-insensitivity
df['community_center'] = df['community_center'].astype(str).str.lower().str.strip()
df['community_center'] = df['community_center'].replace({
    'woodlawn': 0, 
    'hyde park': 1, 
    'hydepark': 1,
    'hyde_park':1,
    'hyd park':1,
})

# 8. Convert "female" to 1 and "male" to 0
df['female'] = df['female'].astype(str).str.lower().str.strip()
df['female'] = df['female'].replace({'female': 1, 'male': 0})
# Ensure the column is numeric integer
df['female'] = pd.to_numeric(df['female']).astype(int)

# 9 & 10. Process Education: Lowercase and convert to 1/0 dummy variables
df['education'] = df['education'].str.lower().str.strip()
# dtype=int ensures the dummies are 1/0 instead of True/False
edu_dummies = pd.get_dummies(df['education'], dtype=int)
edu_cols = ["less than high school", "high school", "higher degree"]
# Concatenate specific dummy columns to the main dataframe
df = pd.concat([df, edu_dummies[edu_cols]], axis=1)

# 11 & 12. Process Race/Ethnicity: Lowercase and convert to 1/0 dummy variables
df['race_ethnicity'] = df['race_ethnicity'].str.lower().str.strip()
race_dummies = pd.get_dummies(df['race_ethnicity'], dtype=int)
race_cols = ["white", "black", "hispanic"]
# Concatenate specific dummy columns to the main dataframe
df = pd.concat([df, race_dummies[race_cols]], axis=1)

# Reorder rows based on the subject_id column in ascending order (from 1, 2, 3...)
df = df.sort_values(by='subject_id')

# Final step: Save the cleaned data to a new CSV file
output_filename = 'cleaned_data_forQ3_Q9.csv'
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("\nCleaning process complete.")
print(f"Final N: {len(df)}")
print(f"File saved as: {output_filename}")

Start N: 973
1. After removing missing values: 930
2. After removing negative age: 876
3. After removing hours > 100: 840
4. After removing BMI < 1.0: 815
5. After removing invalid treatment codes: 815

Cleaning process complete.
Final N: 815
File saved as: cleaned_data_forQ3_Q9.csv


In [6]:
# ====Check the cleaned data====
# Verify that IDs with missing values in the original datasets are correctly excluded from the final cleaned data

# --prompt--
# 以下の4つのファイルをリストとしてループ処理。
# woodlawn.csv, hyde_park.csv, outcome_data.csv, treatment.csv
# 各ファイル内で、いずれかのカラム（列）に1つでも欠損値（NaN）がある行の subject_id をすべて抽出し、重複のないセットにまとめて。
# ただし、subject_id 自体が欠損している行はIDで追跡できないため、別途 outcome_data.csv において subject_id が欠損している行数のみをカウントして。
# cleaned_data_forQ3_Q9.csv を読み込み、そこに含まれる実際のIDを取得。
# 1から1000までのID範囲を定義してください。
# この1〜1000の範囲の中で、最終データに残っていないIDを「ドロップされたID」として特定
# 「1つのカラムでも欠損値があったID」が、すべて「最終データからドロップされたID」に含まれているかを確認して。
# 以下の情報を表示
# 元のファイルで欠損値があったIDのリストと合計数
# ID自体が欠損していた件数（追跡不能分）
# 1〜1000の範囲から実際にドロップされたIDの数とリスト
# 検証結果「すべての欠損値IDが正しく削除された」、またはまだデータに残っている問題のあるIDのリストを表示
# ----

import pandas as pd

# 1. Load the raw datasets to find IDs with missing values
raw_files = ['woodlawn.csv', 'hyde_park.csv', 'outcome_data.csv', 'treatment.csv']
ids_with_missing_values = set()

for file in raw_files:
    df_raw = pd.read_csv(file)
    # Identify rows where any column has a NaN value
    # We extract the 'subject_id' of these rows
    # .dropna(subset=['subject_id']) ensures we only collect IDs that actually exist
    m_ids = df_raw[df_raw.isnull().any(axis=1)]['subject_id'].dropna().unique()
    ids_with_missing_values.update(m_ids)

# 2. Identify IDs in outcome_data where subject_id itself was NaN (if any)
# These cannot be tracked by ID, so we just count them
df_outcome = pd.read_csv('outcome_data.csv')
nan_id_count = df_outcome['subject_id'].isnull().sum()

# 3. Load the final cleaned dataset (which should be a subset of 1-1000)
df_cleaned = pd.read_csv('cleaned_data_forQ3_Q9.csv')
actual_ids = set(df_cleaned['subject_id'].unique())

# 4. Identify which IDs from the 1-1000 range are missing in the final data
expected_range = set(range(1, 1001))
dropped_ids = expected_range - actual_ids

# 5. Cross-check: Are the "missing value IDs" included in the "dropped IDs"?
# This tells us if the people with missing data were successfully removed
confirmed_drops = ids_with_missing_values.intersection(dropped_ids)

# 6. Display the Report
print(f"--- Data Integrity Report ---")
print(f"IDs with missing values in raw files: {sorted(list(ids_with_missing_values))}")
print(f"Total count of IDs with missing values: {len(ids_with_missing_values)}")
print(f"Note: {nan_id_count} records had missing subject_ids entirely (untrackable).")

print(f"\nTotal IDs dropped from the 1-1000 range: {len(dropped_ids)}")
print(f"Dropped IDs: {sorted(list(dropped_ids))}")

print(f"\n--- Verification ---")
if ids_with_missing_values.issubset(dropped_ids):
    print("✅ Success: All IDs identified with missing values were removed from the final dataset.")
else:
    remaining_problem_ids = ids_with_missing_values - dropped_ids
    print(f"⚠️ Warning: Some IDs with missing values are still in the final data: {remaining_problem_ids}")

--- Data Integrity Report ---
IDs with missing values in raw files: [np.int64(65), np.int64(66), np.int64(104), np.int64(134), np.int64(149), np.int64(168), np.int64(183), np.int64(202), np.int64(233), np.int64(236), np.int64(272), np.int64(276), np.int64(305), np.int64(312), np.int64(313), np.int64(326), np.int64(355), np.int64(476), np.int64(487), np.int64(558), np.int64(576), np.int64(597), np.int64(630), np.int64(708), np.int64(820), np.int64(890), np.int64(941)]
Total count of IDs with missing values: 27
Note: 2 records had missing subject_ids entirely (untrackable).

Total IDs dropped from the 1-1000 range: 185
Dropped IDs: [16, 17, 26, 33, 34, 38, 42, 58, 59, 62, 65, 66, 67, 83, 86, 101, 103, 104, 105, 114, 126, 129, 134, 139, 145, 149, 154, 155, 157, 164, 167, 168, 183, 184, 185, 189, 190, 201, 202, 209, 211, 214, 233, 236, 237, 241, 246, 248, 261, 262, 269, 272, 276, 287, 288, 296, 298, 299, 305, 312, 313, 317, 326, 337, 343, 352, 353, 355, 362, 371, 381, 397, 401, 402, 403, 4

In [21]:
import pandas as pd

# List of target files
files = ['woodlawn.csv', 'hyde_park.csv', 'outcome_data.csv', 'treatment.csv']

for file in files:
    df = pd.read_csv(file)
    
    print(f"--- Missing Values in {file} ---")
    
    # Identify missing values with isnull() and calculate the sum per column
    missing_counts = df.isnull().sum()
    
    # Filter and display only columns that have at least one missing value for better readability
    print(missing_counts[missing_counts > 0])
    

--- Missing Values in woodlawn.csv ---
race_ethnicity    10
dtype: int64
--- Missing Values in hyde_park.csv ---
race_ethnicity    17
dtype: int64
--- Missing Values in outcome_data.csv ---
subject_id    2
hours         2
dtype: int64
--- Missing Values in treatment.csv ---
Series([], dtype: int64)
